# List of all Dota 2 emoticons as GIFs and unicode characters

This notebook is a fork of [ListEmoticons.ipynb](https://github.com/rossengeorgiev/dota2_notebooks/blob/master/List%20Emoticons.ipynb) by [rossengeorgiev](https://github.com/rossengeorgiev).

To see all emoticons - just visit the page 

# https://aluerie.github.io/Dota2Utils/ListEmoticons/

---

Nevertheless, The code below generates a list of all emoticons in dota and their unicode character. You can copy & paste the character into Dota 2 to use the emoticon. It doesn't matter if you don't own the emoticon pack.

### Required imports

In [2]:
import os
import vdf
import vpk
import shutil
from datetime import datetime

from typing import TypedDict, Dict

from PIL import Image
from IPython.core.display import HTML
from tqdm import tqdm


out_temp = "./.temp"
out_dir = "./gifs"
out_sizes = [96]  # [32, 96]

### Get .vtex_c files into ./.temp/cache_vtex from your Dota 2 installation (Option 1)

Now let's get list of emoticons from your local Dota 2 installation. 

In [3]:
pak1 = vpk.open("C:/Program Files (x86)/Steam/steamapps/common/dota 2 beta/game/dota/pak01_dir.vpk")

emoticons_definitions = vdf.loads(pak1["scripts/emoticons_definitions.txt"].read().decode("utf-8"))

class EmoteDict(TypedDict):
    image_name: str
    ms_per_frame: str
    aliases: Dict[str, str]

class EmoticonsCategory(TypedDict):
    utf: str
    path: str
    emoticons: Dict[str, EmoteDict]

data: Dict[str, EmoticonsCategory] = {}

for emote_name, vdf_path in emoticons_definitions['emoticons_definitions'].items():
    utf: str = "utf-16le" if emote_name == 'default' else "utf-8" # imagine using utf-16le for just one old file :c 
    temp = vdf.loads(pak1[vdf_path].read().decode(utf))
    temp_emoticons = temp['emoticons']
    data[emote_name] = {
        'utf': utf,
        'path': vdf_path,
        'emoticons': temp_emoticons
    }

In [4]:
if not os.path.isdir(out_temp):
    os.mkdir(out_temp)
if not os.path.isdir(f"{out_temp}/cache_vtex"):
    os.mkdir(f"{out_temp}/cache_vtex")

for emote_name, category in data.items():

    cache_path = f"{out_temp}/cache_vtex/{emote_name}"
    if not os.path.isdir(cache_path):
        os.mkdir(cache_path)

    for k, v in category['emoticons'].items():

        emote_name = v["image_name"].replace(".", "_") + ".vtex_c"
        emoticon_path = f"panorama/images/emoticons/{emote_name}"

        try:
            subfolders = emote_name.rsplit('/', 1)
            if len(subfolders) > 1:
                os.makedirs(f"{cache_path}/{subfolders[0]}")
        except Exception as e:
            pass  # TODO: Better handling

        try:
            save_path = f"{cache_path}/{emote_name}"
            pak1[emoticon_path].save(save_path)
        except:
            print(f"Missing: {emoticon_path}")

### Get .vtex_c files into ./.temp/cache_vtex using SteamCTL (Option 2)

Instead of getting emotes from local installation you can get the emoticons using [SteamCTL](https://pypi.org/project/steamctl/) ([GitHub link](https://github.com/ValvePython/steamctl)). Install it with 
```
pip install steamctl
```
And in the root of iPython notebook use these commands in Windows Terminal (it will prompt to login into steam)  to get a list of all emoticons and all `.vtex_c` files:
```ps1
steamctl depot download -a 570 --vpk -n '*pak01_dir*:*emoticons.txt' -nd -o ./.temp/
steamctl depot download -a 570 --vpk -n '*pak01_dir*:*emoticons/*.vtex_c' -nd -o ./.temp/cache_vtex_c
```

But if you have Dota 2 installed and successfully run the cell above then there is no need in `steamctl`.

In [5]:
# If you decide to use steamctl method then uncomment the following line:
# data = vdf.loads(open(f'{out_temp}/emoticons.txt', 'rb').read().decode('utf-16le'))
# i didnt code the part which grabs all required files with steamctl like ranked/fancontent emotes 

### Decompile .vtex_c files into .png series of frames

Download VRF "CLI Decompiler for windows-x64" from https://vrf.steamdb.info/ and put the path to it into the following cell after exclamation mark (`!`). 

In [6]:
if not os.path.isdir(f"{out_temp}/cache_png"):
    os.mkdir(f"{out_temp}/cache_png")

!"C:/Program Portable/VRF CLI Decompiler/Decompiler.exe" -i "$out_temp/cache_vtex" -o "$out_temp/cache_png" --recursive > NUL

> **Warning**

`Decompiler.exe` might ignore `-o` flag and send all pictures into root of D: disk. I think it happens when `.exe` is not on disk `C:`. Maybe a bug? Anyway, if that happens then we have to fix it ourself.

So if that happens, uncomment the following cell and move files to our desired `$out_temp/cache_png`.

In [7]:
# source = "D:\\"
# dest = "D:\\LAPTOP\\Dota2Utils\\ListEmoticons\\.temp\\cache"

# files = os.listdir(source)
# img_extensions = (".jpg", ".gif", ".png")
# for f in files:
#     if f.endswith(img_extensions):
#         shutil.move(source + f, dest)

In [8]:
# create ./gifs folder
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)

### Generate .gif files combining .png frame-series from the previous step

For the next cell install ImageMagick from https://imagemagick.org/script/download.php#windows 
* At the time, I used portable `ImageMagick-7.1.1-8-portable-Q16-HDRI-x64.zip` edition myself.

> **Warning**
> If you are looking at the following cell from github then if it is not showing correctly then the reason is the strings in the code have HTML elements which GitHub just eats. Download Notebook file itself instead of copypasting it from GitHub page.

The script creates `.gif`-files that are missing from `gifs` folder. 

It also creates `resp` which presents HTML tables containing all the emotes.

In [9]:
table_headers = "<table><tr>" + f"<th colspan={len(out_sizes)}>preview</th><th>chr</th><th>alias</th>" * 3 + "</tr>\n"

table_headers = '''
| preview | chr | alias | preview | chr | alias | preview | chr | alias |
| ---     | --- | ---   | ---     | --- | ---   |     --- | --- |   --- |
'''

def clear_temp_from_pngs_for_gif():
    """
    We are saving .png frame files into temp but we need to clean it up bcs
    if gif 1 has 40 frames and gif 2 has 30 frames. 
    the tool wont remove last 10 frames from gif1 by itself. 
    """
    for x in os.listdir(out_temp):
        if not x.endswith(".png"):
            continue
        os.remove(os.path.join(out_temp, x))

global_counter = 0
counter = 0

resp = ''

for name, category in tqdm(data.items()):

    resp += f'## {name.capitalize()}\n'
    resp += table_headers
    
    for counter, (k, v) in tqdm(enumerate(category['emoticons'].items()), leave=False):
        emote_name = f"{v['image_name'].replace('.', '_')}.png"
        try:
            f = open(f"{out_temp}/cache_png/{name}/{emote_name}", 'rb')
        except IOError:
            print(emote_name, 'IOError')  # TODO: Better handling
            continue
        out_paths = [f"{out_dir}/{int(k):0>3d}_{size:d}.gif" for size in out_sizes]

        # if a gif doesn't exist generate it
        if not all(map(os.path.isfile, out_paths)):  
            clear_temp_from_pngs_for_gif()

            # split the png sequence in seperate files                              
            im = Image.open(f)
            for out_path, size in zip(out_paths, out_sizes):
                for i in range(im.size[0] // 32):
                    im.crop((i * 32, 0, (i + 1) * 32, 32))\
                    .resize((size, size), Image.NEAREST)\
                    .save(f"{out_temp}/{i:0>3d}.png")
                # combines the sequence images into a gif using ImageMagick
                !"C:/Program Portable/ImageMagick/convert" -loop 0 -delay 10 -alpha set -dispose background "$out_temp/*.png" "$out_path"
        
        # generate table cells for emoticon
        if counter % 3 == 0:
            resp += ""    
        for path in out_paths:
            resp += f"| ![emoticon]({path})"
        resp += f"| {chr(0xE000 + int(k)):s} | `:{v['aliases']['0']:s}:`"
        if counter % 3 == 2:
            resp += " |\n"

    clear_temp_from_pngs_for_gif()
    if counter % 3 != 2:
        resp += " |\n"
    resp += "\n"

    global_counter += counter + 1

print('❤️❤️❤️ DONE!!! ❤️❤️❤️')

100%|██████████| 233/233 [00:01<00:00, 163.59it/s]

❤️❤️❤️ DONE!!! ❤️❤️❤️


### Build README.md page

This page is used in GitHub pages, so it's really important to compile it.

In [10]:
today_stats = f"""
<table>
      <tr><td>Last Updated</td><td>{datetime.today().strftime("%d/%B/%Y")}</td></tr>
      <tr><td>Amount of emoticons listed</td><td>{global_counter}</td></tr>
</table>

"""

with open("./README.md", "r", encoding="utf-8") as file:
    file_lines = file.readlines()
    old_lines = []
    for line in file_lines:
        if not line.startswith("## Table of Dota 2 Emoticons"):
            old_lines.append(line)
        else:
            old_lines.append("## Table of Dota 2 Emoticons\n")
            old_lines.append(today_stats)
            break

with open("./README.md", "w", encoding="utf-8") as file:
    file.writelines(old_lines)

with open("./README.md", "a", encoding="utf-8") as file:
    file.write(resp)

### Practice zone

Well, you can still play more with the data.

In [50]:
print(f"Amount of emoticons listed: {global_counter}")
# HTML(resp)

Amount of emoticons listed: 1487
